In [22]:

from pathlib import Path

import numpy as np
import pandas as pd
from scipy import stats, signal
from scipy.interpolate import interp1d
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, silhouette_score, davies_bouldin_score
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import umap

In [ ]:
def parse_parameters(file_path):
    """
    Читает файл parameters.txt и возвращает словарь с нужными параметрами
    """
    params = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.split()
            if len(parts) >= 2:
                key = parts[0]
                value = ' '.join(parts[1:])  # на случай, если значение содержит пробелы
                params[key] = value
    # Извлекаем нужные параметры
    result = {}
    for param in ['PName', 'Msw', 'XUVInt', 'H2a', 'Helium']:
        result[param] = params.get(param)
    return result

def process_files_in_folder_with_params(root_folder):
    """
    Рекурсивно проходит по папке, читает параметры и файлы с данными
    Возвращает список словарей, где каждый словарь — это один файл
    """
    dataset = []
    root = Path(root_folder)
    for data_file_path in root.rglob('*[.txt,.dat]'):
        if data_file_path.name == 'parameters.txt':
            continue  # Пропускаем файл параметров

        # Ищем файл parameters.txt в той же папке
        param_file_path = data_file_path.parent / 'parameters.txt'

        if param_file_path.exists():
            params = parse_parameters(param_file_path)
            # print(f"Файл: {data_file_path}")
            # print(f"Параметры: {params}")
        else:
            # print(f"Файл parameters.txt не найден для {data_file_path}")
            params = {}

        # Читаем V и FullAbs из data файла
        try:
            V, FullAbs = np.loadtxt(data_file_path, usecols=(0, 1), unpack=True, skiprows=1)
            entry = {
                'V': V,
                'FullAbs': FullAbs,
                'PName': params.get('PName'),
                'Msw': params.get('Msw'),
                'XUVInt': params.get('XUVInt'),
                'H2a': params.get('H2a'),
                'Helium': params.get('Helium'),
                # 'File': str(data_file_path)  # можно убрать, если не нужно
            }
            dataset.append(entry)
        except Exception as e:
            print(f"Ошибка при загрузке данных из {data_file_path}: {e}")
        # print("-" * 50)

    return dataset

def compute_basic_stats(arr):
    return {
        'min': np.min(arr),
        'max': np.max(arr),
        'mean': np.mean(arr),
        'median': np.median(arr),
        'std': np.std(arr),
        'var': np.var(arr),
        'skew': stats.skew(arr),
        'kurt': stats.kurtosis(arr)
    }

def compute_peak_features(x, y):
    # global peak
    idx_max = np.argmax(y)
    peak_x = x[idx_max]
    peak_y = y[idx_max]
    # FWHM: find x positions where y >= half max
    half = peak_y / 2.0
    # interpolate to get better precision
    f = interp1d(x, y - half, kind='linear')
    try:
        roots = np.sort(np.unique(np.hstack((np.where(np.sign(y-half)==0)[0]))))
        # simpler approach: find approximate left/right crossing by index
        left_idx = np.where(y[:idx_max] <= half)[0]
        right_idx = np.where(y[idx_max:] <= half)[0]
        if left_idx.size>0:
            left = np.interp(half, y[left_idx[-1]:left_idx[-1]+2], x[left_idx[-1]:left_idx[-1]+2])
        else:
            left = x[0]
        if right_idx.size>0:
            rbase = idx_max + right_idx[0]
            right = np.interp(half, y[rbase:rbase+2], x[rbase:rbase+2]) if rbase+1 < len(x) else x[-1]
        else:
            right = x[-1]
        fwhm = right - left
    except Exception:
        # fallback discrete
        indices = np.where(y >= half)[0]
        if indices.size>0:
            fwhm = x[indices[-1]] - x[indices[0]]
        else:
            fwhm = np.nan
    return {
        'peak_x': float(peak_x),
        'peak_y': float(peak_y),
        'fwhm': float(fwhm)
    }

def compute_spectral_features(y, ncoef=6):
    # FFT amplitude (take first ncoef after DC)
    Y = np.fft.rfft(y - np.mean(y))
    amp = np.abs(Y)
    amps = amp[1:ncoef+1] if len(amp) > ncoef+1 else np.pad(amp[1:], (0,ncoef-len(amp)+1))
    centroid = np.sum(np.arange(len(amp))*amp)/np.sum(amp) if np.sum(amp)>0 else 0
    return {f'fft_{i+1}': float(amps[i]) for i in range(ncoef)}, {'spec_centroid': float(centroid)}


def compute_derivative_features(x, y):
    dx = np.gradient(x)
    dy = np.gradient(y, dx)
    ddy = np.gradient(dy, dx)
    return {
        'dy_mean': float(np.mean(dy)),
        'dy_std': float(np.std(dy)),
        'dy_max': float(np.max(dy)),
        'ddy_mean': float(np.mean(ddy)),
        'sign_changes': int(np.sum(np.diff(np.sign(dy)) != 0))
    }


def compute_autocorr_features(y, lags=[1,2,5]):
    res = {}
    y0 = y - np.mean(y)
    var = np.var(y0)
    for lag in lags:
        if lag < len(y0):
            res[f'autocorr_lag{lag}'] = float(np.corrcoef(y0[:-lag], y0[lag:])[0,1])
        else:
            res[f'autocorr_lag{lag}'] = np.nan
    return res


def extract_features_from_profile(x, y):
    feats = {}
    feats.update({'len': len(y)})
    feats.update({f'Q_{q}': float(np.percentile(y, q)) for q in [1,5,25,50,75,95,99]})
    feats.update(compute_basic_stats(y))
    feats.update(compute_peak_features(x, y))
    fft_feats, spec = compute_spectral_features(y, ncoef=6)
    feats.update(fft_feats)
    feats.update(spec)
    feats.update(compute_derivative_features(x, y))
    feats.update(compute_autocorr_features(y, lags=[1,2,5]))
    # AUC
    feats['auc'] = float(np.trapz(y, x))
    # left/center/right areas relative to peak
    idx_max = np.argmax(y)
    left_area = float(np.trapz(y[:idx_max+1], x[:idx_max+1])) if idx_max>0 else 0.0
    right_area = float(np.trapz(y[idx_max:], x[idx_max:])) if idx_max < len(y)-1 else 0.0
    feats['left_area'] = left_area
    feats['right_area'] = right_area
    feats['center_area_ratio'] = (feats['auc'] - left_area - right_area) / feats['auc'] if feats['auc']>0 else 0.0
    return feats


def analyze_dataframe(df):
    """
    Принимает DataFrame с колонками V, FullAbs и параметрами.
    Возвращает новый DataFrame: один объект = один профиль со всеми признаками.
    """
    rows = []

    for idx, row in df.iterrows():
        x = np.array(row['V'])
        y = np.array(row['FullAbs'])

        # Извлекаем признаки профиля
        feats = extract_features_from_profile(x, y)

        # Добавляем исходные параметры
        feats['Msw'] = row.get('Msw')
        feats['XUVInt'] = row.get('XUVInt')
        
        feats['H2a'] = row.get('H2a')
        feats['Helium'] = row.get('Helium')

        rows.append(feats)

    return pd.DataFrame(rows)

In [12]:
folder_path = '/home/kirill/diploma/dataset'  # Укажите путь к вашей папке
dataset = process_files_in_folder_with_params(folder_path)
dataset

[{'V': array([-5. , -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4.3, -4.2, -4.1, -4. ,
         -3.9, -3.8, -3.7, -3.6, -3.5, -3.4, -3.3, -3.2, -3.1, -3. , -2.9,
         -2.8, -2.7, -2.6, -2.5, -2.4, -2.3, -2.2, -2.1, -2. , -1.9, -1.8,
         -1.7, -1.6, -1.5, -1.4, -1.3, -1.2, -1.1, -1. , -0.9, -0.8, -0.7,
         -0.6, -0.5, -0.4, -0.3, -0.2, -0.1,  0. ,  0.1,  0.2,  0.3,  0.4,
          0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,  1.2,  1.3,  1.4,  1.5,
          1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,  2.3,  2.4,  2.5,  2.6,
          2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,  3.4,  3.5,  3.6,  3.7,
          3.8,  3.9,  4. ]),
  'FullAbs': array([5.06662e-05, 1.03927e-04, 1.41210e-04, 1.90025e-04, 2.53660e-04,
         3.35966e-04, 4.40836e-04, 5.71427e-04, 7.29096e-04, 9.12232e-04,
         1.11527e-03, 1.32825e-03, 1.53720e-03, 1.72550e-03, 1.87607e-03,
         1.97396e-03, 2.00890e-03, 1.97714e-03, 1.88221e-03, 1.73429e-03,
         1.54834e-03, 1.34154e-03, 1.13071e-03, 9.30033e-04

### проскалировать target переменные
### optuna для перебора гиперпараметров

In [13]:
df = pd.DataFrame(dataset)
print(len(df))
df

204


,V,FullAbs,PName,Msw,XUVInt,H2a,Helium
0,"[-5.0, -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4....","[5.06662e-05, 0.000103927, 0.00014121, 0.00019...",23,1e11,10,0,0.075022
1,"[-5.0, -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4....","[8.43054e-06, 2.00702e-05, 3.35282e-05, 5.4568...",23,1e11,10,0,0.075022
2,"[-4.0, -3.8, -3.6, -3.4, -3.2, -3.0, -2.8, -2....","[0.0185282, 0.02618, 0.0332259, 0.0359916, 0.0...",WASP107b,1e11,10,1,0.075022
3,"[-4.0, -3.9, -3.8, -3.7, -3.6, -3.5, -3.4, -3....","[0.00491718, 0.00597564, 0.00724545, 0.0086960...",WASP107b,2e13,10,0,0.005022
4,"[-4.0, -3.8, -3.6, -3.4, -3.2, -3.0, -2.8, -2....","[0.00514849, 0.0083293, 0.0147668, 0.0216009, ...",WASP107b,2e13,10,0,0.005022
...,...,...,...,...,...,...,...
199,"[-5.0, -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4....","[0.0119604, 0.0216392, 0.0254084, 0.0309585, 0...",WASP107b,61e12,47,0,0.184022
200,"[-5.0, -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4....","[0.00599491, 0.0103018, 0.0110272, 0.0122752, ...",WASP107b,17e12,49,0,0.040022
201,"[-5.0, -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4....","[0.0117802, 0.0209631, 0.0239166, 0.0283212, 0...",WASP107b,59e12,49,0,0.105022
202,"[-5.0, -4.9, -4.8, -4.7, -4.6, -4.5, -4.4, -4....","[0.00145743, 0.0026211, 0.00305922, 0.00371948...",WASP107b,85e11,7,0,0.015022


In [40]:
df1 = analyze_dataframe(df)
df1['XUVInt'] = df1['XUVInt'].astype(float)
df1.drop('len', axis=1, inplace=True)
df1.head()

/home/kirill/.cache/pypoetry/virtualenvs/bachalor-diploma-Fn-aLXeR-py3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:1309: RuntimeWarning: divide by zero encountered in divide
  a = -(dx2) / (dx1 * (dx1 + dx2))
/home/kirill/.cache/pypoetry/virtualenvs/bachalor-diploma-Fn-aLXeR-py3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:1309: RuntimeWarning: invalid value encountered in divide
  a = -(dx2) / (dx1 * (dx1 + dx2))
/home/kirill/.cache/pypoetry/virtualenvs/bachalor-diploma-Fn-aLXeR-py3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:1310: RuntimeWarning: divide by zero encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/home/kirill/.cache/pypoetry/virtualenvs/bachalor-diploma-Fn-aLXeR-py3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:1310: RuntimeWarning: invalid value encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/home/kirill/.cache/pypoetry/virtualenvs/bachalor-diploma-Fn-aLXeR-py3.13/lib/python3.13

,Q_1,Q_5,Q_25,Q_50,Q_75,Q_95,Q_99,min,max,mean,...,autocorr_lag2,autocorr_lag5,auc,left_area,right_area,center_area_ratio,Msw,XUVInt,H2a,Helium
0,0.000001,0.000003,0.000234,0.000908,0.002557,0.013117,0.014264,8.437680e-07,0.014288,0.002761,...,0.961785,0.774837,0.025124,0.013225,0.011899,-6.904615e-17,1e11,10.0,0,0.075022
1,0.000003,0.000005,0.000088,0.000537,0.001767,0.010691,0.011674,2.995720e-06,0.011691,0.002138,...,0.958755,0.758079,0.019451,0.011138,0.008314,0.000000e+00,1e11,10.0,0,0.075022
2,0.014112,0.019955,0.033191,0.066578,0.096147,0.112992,0.113990,1.116720e-02,0.114435,0.066150,...,0.887453,0.479351,0.539463,0.200936,0.338528,0.000000e+00,1e11,10.0,1,0.075022
3,0.000039,0.000112,0.002791,0.006771,0.014111,0.062292,0.067370,3.061450e-05,0.068534,0.014002,...,0.945354,0.703919,0.113165,0.062959,0.050206,0.000000e+00,2e13,10.0,0,0.005022
4,0.000212,0.000596,0.006109,0.012540,0.022449,0.103030,0.113731,1.537870e-04,0.117326,0.023792,...,0.768846,0.172645,0.194568,0.101704,0.092864,0.000000e+00,2e13,10.0,0,0.005022


In [41]:
print(df1.columns)

Index(['Q_1', 'Q_5', 'Q_25', 'Q_50', 'Q_75', 'Q_95', 'Q_99', 'min', 'max',
       'mean', 'median', 'std', 'var', 'skew', 'kurt', 'peak_x', 'peak_y',
       'fwhm', 'fft_1', 'fft_2', 'fft_3', 'fft_4', 'fft_5', 'fft_6',
       'spec_centroid', 'dy_mean', 'dy_std', 'dy_max', 'ddy_mean',
       'sign_changes', 'autocorr_lag1', 'autocorr_lag2', 'autocorr_lag5',
       'auc', 'left_area', 'right_area', 'center_area_ratio', 'Msw', 'XUVInt',
       'H2a', 'Helium'],
      dtype='object')


In [42]:
OUTDIR = Path('plots')
def ensure_numeric_targets(df, target_cols):
    df2 = df.copy()
    for c in target_cols:
        if c in df2.columns:
            # try convert strings like '1e11' -> float
            df2[c] = pd.to_numeric(df2[c], errors='coerce')
    return df2

def regression_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    medae = np.median(np.abs(y_true - y_pred))
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MedianAE': medae}

def print_metrics_df(metrics_dict):
    dfm = pd.DataFrame(metrics_dict).T
    display(dfm.round(5))
    return dfm

def prepare_data(df, feature_cols=None, target_cols=["Msw", 'XUVInt', 'H2a', "Helium"], dropna=True, scaler='robust'):
    """
    Возвращает X, y, colnames и scaler fitted.
    Если feature_cols None, берет все колонки кроме target_cols.
    """
    df = df.copy()
    if target_cols is None:
        raise ValueError("target_cols required")
    df = ensure_numeric_targets(df, target_cols)
    if dropna:
        df = df.dropna(subset=target_cols)  # drop rows where targets missing
    if feature_cols is None:
        feature_cols = [c for c in df.columns if c not in target_cols and c != 'File' and c != 'V' and c != 'FullAbs']
    X = df[feature_cols].copy()
    y = df[target_cols].copy()

    
    def safe_to_scalar(v):
            if isinstance(v, (list, tuple, np.ndarray)):
                try:
                    return float(np.mean(np.array(v, dtype=float)))
                except:
                    return np.nan
            if isinstance(v, str):
                try:
                    return float(v)
                except:
                    return np.nan
            if np.isscalar(v):
                try:
                    return float(v)
                except:
                    return np.nan
            return np.nan
    # convert any array-like cells to numeric summaries if present (defensive)
    for c in X.columns:
            # try to convert arrays stored as lists -> take mean as fallback
        X[c] = X[c].apply(safe_to_scalar)
    X = X.astype(float)

    y = df[target_cols]
    # scaler
    if scaler == 'robust':
        scaler_obj = RobustScaler()
    else:
        scaler_obj = StandardScaler()
    X_scaled = pd.DataFrame(scaler_obj.fit_transform(X.fillna(0)), columns=feature_cols, index=X.index)
    return X_scaled, y, feature_cols, scaler_obj

def plot_correlation_matrix(X, y, outdir=OUTDIR):
    df = pd.concat([X, y], axis=1)
    corr = df.corr()
    plt.figure(figsize=(14,12))
    sns.heatmap(corr, cmap='RdBu_r', center=0)
    plt.title('Correlation matrix (features + targets)')
    plt.tight_layout()
    plt.savefig(outdir/"corr_matrix.png", dpi=150)
    plt.close()
    return corr

def plot_pair_scatter(X, y, features=None, outdir=OUTDIR):
    if features is None:
        features = list(X.columns)[:6]
    df = pd.concat([X[features], y.reset_index(drop=True)], axis=1)
    sns.pairplot(df.sample(min(200, len(df))), corner=True)
    plt.savefig(outdir/"pairplot_sample.png", dpi=150)
    plt.close()

In [45]:
# -------------------------
# 3) Clustering + DR (PCA + UMAP + t-SNE (optional))
# -------------------------
def clustering_and_visualization(X, y, n_clusters=3, outdir=OUTDIR):
    pca = PCA(n_components=min(10, X.shape[1]))
    X_pca = pca.fit_transform(X)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    labels = kmeans.fit_predict(X_pca[:,:min(5, X_pca.shape[1])])
    sil = silhouette_score(X_pca[:,:min(5, X_pca.shape[1])], labels)
    db = davies_bouldin_score(X_pca[:,:min(5, X_pca.shape[1])], labels)
    print(f"KMeans clusters={n_clusters} silhouette={sil:.3f} DB={db:.3f}")

    # UMAP visualization
    reducer = umap.UMAP(n_components=2, random_state=42)
    embedding = reducer.fit_transform(X)
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=labels, palette='tab10', s=30, alpha=0.8)
    plt.title('UMAP projection colored by KMeans cluster')
    plt.legend(title='cluster')
    plt.tight_layout()
    plt.savefig(outdir/"umap_kmeans.png", dpi=150)
    plt.close()

    # color by a target (example: Msw if present)
    if 'H2a' in y.columns:
        plt.figure(figsize=(8,6))
        sc = plt.scatter(embedding[:,0], embedding[:,1], c=y['H2a'], cmap='viridis', s=30)
        plt.colorbar(sc, label='H2a')
        plt.title('UMAP colored by H2a')
        plt.tight_layout()
        plt.savefig(outdir/"umap_h2a.png", dpi=150)
        plt.close()

    return {'labels': labels, 'pca': pca, 'umap_embedding': embedding, 'silhouette': sil, 'davies_bouldin': db}

# -------------------------
# 4) Regression: test multiple models with CV
# -------------------------
def evaluate_regressors_cv(X, y, models=None, cv=5, scoring=None):
    """
    models: dict name->estimator (single-output or multioutput)
    y: DataFrame with multiple targets -> we'll train separate models per target unless estimator supports multioutput
    returns dict of metrics per model per target (mean CV)
    """
    if scoring is None:
        scoring = {'r2':'r2', 'neg_mse':'neg_mean_squared_error', 'neg_mae':'neg_mean_absolute_error'}
    results = {}
    for name, model in models.items():
        print(f"Evaluating model: {name}")
        results[name] = {}
        # if model handles multioutput (like MultiOutputRegressor), run cross_val_predict? we'll do per-target CV for comparability
        for target in y.columns:
            scores = cross_validate(model, X, y[target], cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False)
            # aggregate
            r2 = np.mean(scores['test_r2'])
            rmse = np.mean(np.sqrt(-scores['test_neg_mse']))
            mae = np.mean(-scores['test_neg_mae'])
            results[name][target] = {'R2': r2, 'RMSE': rmse, 'MAE': mae}
            print(f"  target={target}: R2={r2:.3f}, RMSE={rmse:.3e}, MAE={mae:.3e}")
    return results

# -------------------------
# 5) Fit final models and compute importances + SHAP
# -------------------------
def fit_final_models(X_train, y_train, X_test, y_test, model_factory=None, outdir=OUTDIR):
    """
    model_factory: function(target_name) -> estimator
    Возвращает словарь с обученными моделями и метриками на тесте.
    """
    models = {}
    metrics = {}
    shap_info = {}
    for target in y_train.columns:
        model = model_factory(target)
        model.fit(X_train, y_train[target])
        y_pred = model.predict(X_test)
        metrics[target] = regression_metrics(y_test[target], y_pred)
        models[target] = model
        # permutation importance
        try:
            perm = permutation_importance(model, X_test, y_test[target], n_repeats=20, random_state=0, n_jobs=1)
            fi = pd.DataFrame({'feature': X_train.columns, 'imp_mean': perm.importances_mean, 'imp_std': perm.importances_std})
            fi = fi.sort_values('imp_mean', ascending=False)
            fi.to_csv(outdir/f'perm_importance_{target}.csv', index=False)
            # quick barplot
            plt.figure(figsize=(8,4))
            sns.barplot(x='imp_mean', y='feature', data=fi.head(20))
            plt.title(f'Permutation importance (test) for {target}')
            plt.tight_layout(); plt.savefig(outdir/f'perm_importance_{target}.png', dpi=150); plt.close()
        except Exception as e:
            print("Permutation importance failed:", e)

        # SHAP for tree models
        try:
            if hasattr(model, 'feature_importances_') or model.__class__.__name__.lower().find('xgboost')>=0 or model.__class__.__name__.lower().find('catboost')>=0 or isinstance(model, RandomForestRegressor):
                explainer = shap.TreeExplainer(model)
                shap_vals = explainer.shap_values(X_test)
                shap.summary_plot(shap_vals, X_test, show=False)
                plt.savefig(outdir/f'shap_summary_{target}.png', dpi=150, bbox_inches='tight')
                plt.close()
                shap_info[target] = {'explainer': explainer, 'shap_values': shap_vals}
        except Exception as e:
            print("SHAP failed for", target, e)
    return models, metrics, shap_info

# -------------------------
# 6) Feature selection helpers
# -------------------------
def correlation_feature_prune(X, threshold=0.95):
    """
    Убираем сильно коррелированные фичи (взяли одну из пары).
    """
    corr = X.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return to_drop

def rfe_feature_selection(estimator, X, y, n_features_to_select=20):
    rfe = RFE(estimator=estimator, n_features_to_select=n_features_to_select, step=0.1)
    rfe.fit(X, y)
    support = np.array(X.columns)[rfe.support_]
    ranking = pd.Series(rfe.ranking_, index=X.columns).sort_values()
    return list(support), ranking

# -------------------------
# 7) Pipeline runner
# -------------------------
def run_full_analysis(df, target_cols=['Msw','XUVInt','H2a','Helium'], test_size=0.2, random_state=42):
    # prepare
    X, y, feat_cols, scaler_obj = prepare_data(df, target_cols=target_cols)
    
    print("Features used:", len(feat_cols))
    # EDA
    corr = plot_correlation_matrix(X, y)
    plot_pair_scatter(X, y)

    # clustering
    cluster_res = clustering_and_visualization(X, y, n_clusters=3)

    # models to try
    models = {
        'Ridge': Ridge(),
        'RandomForest': RandomForestRegressor(n_estimators=200, random_state=0)
    }
    if XGBRegressor is not None:
        models['XGBoost'] = XGBRegressor(n_estimators=200, random_state=0, verbosity=0)
    if CatBoostRegressor is not None:
        models['CatBoost'] = CatBoostRegressor(iterations=200, verbose=0, random_state=0)

    # cross-val evaluation (per-target)
    cv_results = evaluate_regressors_cv(X, y, models=models, cv=5)

    # train/test split & final fit
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    def model_factory(target):
        # choose best model by cv results R2 (simple heuristic)
        best_model_name = max(models.keys(), key=lambda nm: cv_results[nm][target]['R2'])
        print(f"  -> using {best_model_name} for {target}")
        chosen = models[best_model_name]
        # if chosen is tree and supports multioutput, clone etc
        from sklearn.base import clone
        return clone(chosen)
    models_fitted, test_metrics, shap_info = fit_final_models(X_train, y_train, X_test, y_test, model_factory=model_factory)

    # feature pruning by correlation
    to_drop = correlation_feature_prune(X, threshold=0.95)
    print("Highly correlated features to drop (threshold 0.95):", to_drop)

    # RFE (example for first target)
    rfe_support, rfe_ranking = rfe_feature_selection(RandomForestRegressor(n_estimators=100, random_state=0), X, y[target_cols[0]], n_features_to_select=min(20, X.shape[1]))
    print("Top features by RFE for target", target_cols[0], rfe_support)

    # save summary metrics
    pd.DataFrame(test_metrics).T.to_csv(OUTDIR/"test_metrics.csv")

    return {
        'X': X, 'y': y, 'cluster_res': cluster_res,
        'cv_results': cv_results, 'models_fitted': models_fitted,
        'test_metrics': test_metrics, 'shap_info': shap_info,
        'to_drop': to_drop, 'rfe_ranking': rfe_ranking
    }

In [46]:
run_full_analysis(df1)

Features used: 37
KMeans clusters=3 silhouette=0.757 DB=0.577


/home/kirill/.cache/pypoetry/virtualenvs/bachalor-diploma-Fn-aLXeR-py3.13/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Evaluating model: Ridge
  target=Msw: R2=0.186, RMSE=1.552e+13, MAE=1.087e+13
  target=XUVInt: R2=-1.861, RMSE=1.577e+01, MAE=1.310e+01
  target=H2a: R2=0.211, RMSE=4.365e-01, MAE=3.679e-01
  target=Helium: R2=0.122, RMSE=4.779e-02, MAE=3.115e-02
Evaluating model: RandomForest
  target=Msw: R2=0.316, RMSE=1.474e+13, MAE=9.308e+12
  target=XUVInt: R2=-0.590, RMSE=1.258e+01, MAE=1.028e+01
  target=H2a: R2=0.327, RMSE=4.043e-01, MAE=3.187e-01
  target=Helium: R2=0.075, RMSE=4.506e-02, MAE=2.842e-02
Evaluating model: XGBoost
  target=Msw: R2=0.212, RMSE=1.584e+13, MAE=9.938e+12
  target=XUVInt: R2=-0.923, RMSE=1.368e+01, MAE=1.093e+01
  target=H2a: R2=0.309, RMSE=4.076e-01, MAE=2.695e-01
  target=Helium: R2=-0.302, RMSE=4.936e-02, MAE=3.073e-02
Evaluating model: CatBoost
  target=Msw: R2=0.463, RMSE=1.365e+13, MAE=8.522e+12
  target=XUVInt: R2=-0.782, RMSE=1.288e+01, MAE=1.060e+01
  target=H2a: R2=0.381, RMSE=3.852e-01, MAE=2.983e-01
  target=Helium: R2=0.147, RMSE=4.291e-02, MAE=2.677e-02

{'X':            Q_1        Q_5      Q_25      Q_50      Q_75      Q_95      Q_99  \
 0    -0.106382  -0.114996 -0.426316 -0.477710 -0.571425 -0.623352 -0.616098   
 1    -0.103024  -0.114126 -0.434448 -0.485762 -0.580922 -0.644793 -0.638229   
 2    21.694037  10.510034  1.409911  0.945333  0.554756  0.259622  0.236065   
 3    -0.048129  -0.056853 -0.283871 -0.350677 -0.432393 -0.188602 -0.162307   
 4     0.219624   0.200675 -0.098991 -0.225656 -0.332058  0.171550  0.233852   
 ..         ...        ...       ...       ...       ...       ...       ...   
 199  -0.018876  -0.068164  1.207381  1.191507  2.445341  3.256081  3.236895   
 200   0.429476   0.608170  0.736817  0.801251  0.775212  0.651948  0.658774   
 201  -0.062529  -0.082176  1.138605  0.777596  1.954416  2.182247  2.160021   
 202   0.083932  -0.001527 -0.163606 -0.271485 -0.212941 -0.126720 -0.095569   
 203   0.903643   0.536511  0.554707  0.530923  1.207789  1.423012  1.443407   
 
            min       max      me